In [39]:
from bs4 import BeautifulSoup
import sys,glob
import os
import pandas as pd
import pickle

## preprocessing TDT4

In [2]:
def extract_topic_rows(file_path="./tdt4_aem_txt/annotations/2002_topics/tdt2002_topic_rel.v2_1", datasplit="train"):
    rows = []
    with open(file_path) as f:
        for line in f:
            if "ONTOPIC" in line:
                splits = line.split()
                topicid, level, docno, fileid = [each.split('=')[-1] for each in splits[1:-1]]
                rows.append([topicid, level, docno, fileid, datasplit])
    return rows

rows_topic2002 = extract_topic_rows("./tdt4_aem_txt/annotations/2002_topics/tdt2002_topic_rel.v2_1", datasplit="train")
rows_topic2003 = extract_topic_rows("./tdt4_aem_txt/annotations/2003_topics/tdt2003_topic_rel.v2_0", datasplit="test")
print(rows_topic2002[:4])
df_topic = pd.DataFrame(rows_topic2002 + rows_topic2003, columns=['topicid', 'level', 'docno', 'fileid', "datasplit"])
df_topic.head()

[['40001', 'YES', 'VOA20010124.2000.2135', '20010124_2000_2100_VOA_ENG', 'train'], ['40001', 'YES', 'ZBN20010126.0400.0019', '20010126_0400_0600_ZBN_MAN', 'train'], ['40001', 'YES', 'CTV20010128.1330.1178', '20010128_1330_1400_CTV_MAN', 'train'], ['40001', 'YES', 'AFA20010121.1000.0029', '20010121_1000_1359_AFP_ARB', 'train']]


,topicid,level,docno,fileid,datasplit
0,40001,YES,VOA20010124.2000.2135,20010124_2000_2100_VOA_ENG,train
1,40001,YES,ZBN20010126.0400.0019,20010126_0400_0600_ZBN_MAN,train
2,40001,YES,CTV20010128.1330.1178,20010128_1330_1400_CTV_MAN,train
3,40001,YES,AFA20010121.1000.0029,20010121_1000_1359_AFP_ARB,train
4,40001,YES,AFA20010125.1800.0196,20010125_1800_2359_AFP_ARB,train


In [63]:
def extract_text_rows(file_path):
    with open(fileName_relative, encoding = "ISO-8859-1") as f:
        file_str = f.read()
        soup = BeautifulSoup(file_str,'html.parser')
        rows_docno, rows_text = soup.find_all("docno"), soup.find_all("text")
        rows_docno = [each.text.split(" ")[1] for each in rows_docno]
#         print(rows_text)
        rows_text = [" ".join(each.text.split(" ")[1:-1]) for each in rows_text]
    return [[a, b] for a, b in zip(rows_docno, rows_text)]


all_rows = []
for fileName_relative in glob.glob("./tdt4_aem_txt/data/tkn_sgm/*",recursive=True):       ## first get full file name with directores using for loop
    
#     print(fileName_relative)
    all_rows.extend(extract_text_rows(fileName_relative))

In [64]:
df_texts = pd.DataFrame(all_rows, columns=["docno", "text"])
df_texts.head()

,docno,text
0,APW20010119.2116.0889,12-year pursuit of peace in the Middle East at...
1,APW20010119.2116.0890,have offered former CIA Director John M. Deutc...
2,APW20010119.2116.0891,surpluses will be big enough to accommodate bo...
3,APW20010119.2116.0892,"a farewell gesture, the Clinton administration..."
4,APW20010119.2116.0893,Dolphins linebacker Zach Thomas and Oakland Ra...


In [71]:
df_texts = df_texts[df_texts.text.str.len() > 5]
df_texts.to_csv("df_texts_large.csv")

In [72]:
print(df_texts.shape)
print(df_topic.shape)

(27362, 2)
(6333, 5)


In [73]:
df_en_news = df_topic.merge(right=df_texts, on="docno")
df_en_news.shape

(1926, 6)

In [74]:
df_en_news['unique_key'] = df_en_news['docno'] + df_en_news['datasplit']
len(set(df_en_news['unique_key']))

1906

In [75]:
df_ = df_en_news[['unique_key', 'topicid']].groupby("unique_key").count()

final_uniq_set = set(df_[df_['topicid'] == 1].index)
print(len(final_uniq_set), final_uniq_set)

1887 {'CNN20010124.1400.0609train', 'NYT20001209.1219.0079train', 'VOA20001129.2000.1967test', 'APW20001108.1217.0841test', 'APW20001111.2025.0544test', 'APW20001226.0519.0148test', 'VOA20001004.2100.0132train', 'APW20001215.2115.0812train', 'ABC20010126.1830.0036train', 'APW20010120.2106.0701train', 'NBC20001006.1830.0045train', 'CNN20001205.1400.0586train', 'VOA20001023.2100.1970train', 'VOA20001211.2000.2124train', 'NYT20010127.1754.0170train', 'APW20010118.0332.0234train', 'ABC20010105.1830.0550train', 'APW20001017.1745.0615train', 'APW20001017.1313.0396test', 'ABC20010125.1830.1292train', 'PRI20001031.2000.0625train', 'CNN20001201.1400.0530train', 'NBC20010129.1830.0501train', 'APW20001022.1003.0254test', 'APW20010110.1238.0410train', 'MNB20001207.2100.1797train', 'CNN20010113.0900.0017train', 'APW20001222.1719.0645test', 'CNN20010120.1400.1150train', 'NYT20010120.1029.0062train', 'CNN20001016.1400.0174train', 'APW20001108.1217.0851test', 'PRI20010116.2000.0652train', 'ABC20010124

In [76]:
df_en_news[['unique_key', 'topicid']].groupby("unique_key").count().sort_values(by="topicid")

,topicid
unique_key,
ABC20001001.1830.0036test,1
NYT20001018.1930.0307train,1
NYT20001017.2211.0361test,1
NYT20001016.0919.0092train,1
NYT20001016.0919.0091train,1
...,...
VOA20001112.2100.0222test,2
APW20001022.1003.0272test,2
NYT20001021.1741.0161train,2


In [77]:
df_en_news[df_en_news.docno == "NYT20001026.1633.0209"]

,topicid,level,docno,fileid,datasplit,text,unique_key
1169,40059,YES,NYT20001026.1633.0209,20001026_1633_1854_NYT_NYT,train,on their state-controlled television stations ...,NYT20001026.1633.0209train
1170,41004,YES,NYT20001026.1633.0209,20001026_1633_1854_NYT_NYT,test,on their state-controlled television stations ...,NYT20001026.1633.0209test
1171,41010,YES,NYT20001026.1633.0209,20001026_1633_1854_NYT_NYT,test,on their state-controlled television stations ...,NYT20001026.1633.0209test
1172,41018,YES,NYT20001026.1633.0209,20001026_1633_1854_NYT_NYT,test,on their state-controlled television stations ...,NYT20001026.1633.0209test


In [78]:
1906-19

1887

In [79]:
# remove all the documents that belong to multiple events, matching "Adaptive online event detection in news streams"
df_en_news = df_en_news[df_en_news.unique_key.isin(final_uniq_set)]

In [80]:
df_en_news['date'] = df_en_news.fileid.str.split("_").str[0].apply(lambda x: x[:4]+'-'+x[4:6]+"-"+x[6:]+" 00:00:00")
df_en_news['id'] = df_en_news.index
df_en_news['cluster'] = df_en_news['topicid']
df_en_news.shape

(1887, 10)

In [81]:
df_en_news.to_csv("processed_tdt4.csv")

In [82]:
df_en_news.head(5)

,topicid,level,docno,fileid,datasplit,text,unique_key,date,id,cluster
0,40001,YES,VOA20010124.2000.2135,20010124_2000_2100_VOA_ENG,train,winds and ocean currents are carrying a massiv...,VOA20010124.2000.2135train,2001-01-24 00:00:00,0,40001
1,40001,YES,PRI20010125.2000.2656,20010125_2000_2100_PRI_TWD,train,"another advantage of hydrogen over oil? Well, ...",PRI20010125.2000.2656train,2001-01-25 00:00:00,1,40001
2,40001,YES,PRI20010125.2000.0231,20010125_2000_2100_PRI_TWD,train,Captain of the stricken tanker off the Galapag...,PRI20010125.2000.0231train,2001-01-25 00:00:00,2,40001
3,40001,YES,VOA20010122.2000.0223,20010122_2000_2100_VOA_ENG,train,President Gustavo Naboi has declared a state o...,VOA20010122.2000.0223train,2001-01-22 00:00:00,3,40001
4,40001,YES,APW20010123.2129.0801,20010123_2129_2213_APW_ENG,train,weather blocked oil cleanup efforts around a s...,APW20010123.2129.0801train,2001-01-23 00:00:00,4,40001


# read to corpus class

In [83]:
class CorpusClass:
    def __init__(self, documents):
        self.documents = documents

In [84]:
df_train = df_en_news[df_en_news.datasplit == "train"]
df_train.shape

(1270, 10)

In [85]:
df_test = df_en_news[df_en_news.datasplit == "test"]
df_test.shape

(617, 10)

In [86]:
def convert_df_to_corpus(df_):
    documents = []
    for row in df_.iterrows():
#         print(row)
#         print(row[1].topicid)
        documents.append({
            "id": row[1]['id'],
            "full_text": row[1]['text'].lower(),
            "lang": "eng",
            "date": row[1]['date'],
            "cluster": row[1]['cluster'],
            "docno": row[1]['docno']
        })
    
    return documents

train_documents = convert_df_to_corpus(df_train)
test_documents = convert_df_to_corpus(df_test)

In [87]:
print(len(train_documents), len(test_documents))

1270 617


In [88]:
train_corpus, test_corpus = CorpusClass(train_documents), CorpusClass(test_documents)

## extract entities (first)
    - run `extract_entities.py`

In [110]:
with open('./train_dev_entities.pickle', 'rb') as handle:
    train_corpus = pickle.load(handle)

with open('./test_entities.pickle', 'rb') as handle:
    test_corpus = pickle.load(handle)

In [113]:
for doc in train_corpus.documents:
    print(doc)
    break

{'id': 0, 'full_text': "winds and ocean currents are carrying a massive oil slick away \nfrom ecuador's galapico's islands. scientists said wednesday the damage \non the islands to rare plant and animal life could have been worse \nif the slick had continued to wash ashore.", 'lang': 'eng', 'date': '2001-01-24 00:00:00', 'cluster': '40001', 'docno': 'VOA20010124.2000.2135', 'features': {'Lemmas_all': {'l_galapico': 0.32831229132315914, 'l_ecuador': 0.2389661221779755, 'l_slick': 0.480664633953581, 'l_ashore': 0.2700018295029832, 'l_ocean': 0.19811616836560575, 'l_animal': 0.17633028567714515, 'l_island': 0.31235515801921876, 'l_plant': 0.15568347849508368, 'l_wind': 0.15701698112610898, 'l_wash': 0.19395199209469, 'l_rare': 0.16608814573040104, 'l_bad': 0.11518475369160017, 'l_damage': 0.14656328713783834, 'l_oil': 0.15027884429765662, 'l_current': 0.13428093787418266, 'l_life': 0.10650595074542996, 'l_wednesday': 0.10527260435746821, 'l_continue': 0.100181632707379, 'l_away': 0.110716

## tf-idf
- entity
- token
- lemma


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Tokens_all

In [90]:
df_texts.text = df_texts.text.str.lower()

In [91]:
df_texts.head()

,docno,text
0,APW20010119.2116.0889,12-year pursuit of peace in the middle east at...
1,APW20010119.2116.0890,have offered former cia director john m. deutc...
2,APW20010119.2116.0891,surpluses will be big enough to accommodate bo...
3,APW20010119.2116.0892,"a farewell gesture, the clinton administration..."
4,APW20010119.2116.0893,dolphins linebacker zach thomas and oakland ra...


In [92]:
vectorizer = TfidfVectorizer(stop_words={"English"}, max_features=50000)
tfidf_matrix = vectorizer.fit_transform(df_texts.text)
feature_names = vectorizer.get_feature_names()


/home/hjian42/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['english'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [93]:
df_texts.shape

(27362, 2)

In [96]:
doc2tfidf = {} # docno, tfidf_dict
docno_set = set(df_en_news.docno)

for doc in range(len(df_texts)):
#     print(doc)
    docno = df_texts.docno.values[doc]
    if docno in docno_set:
        feature_index = tfidf_matrix[doc,:].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
        tf_idf_dict = {"t_"+ feature_names[i]: s for (i, s) in tfidf_scores}
        doc2tfidf[docno] = tf_idf_dict

In [114]:
for doc in train_corpus.documents:
    doc['features']["Tokens_all"] = doc2tfidf[doc['docno']]
for doc in test_corpus.documents:
    doc['features']["Tokens_all"] = doc2tfidf[doc['docno']]

In [115]:
len(set(df_en_news.docno)) # Note: train and test share some common files

1867

In [107]:
# with open('./train_dev.pickle', 'wb') as handle:
#     pickle.dump(train_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('./test.pickle', 'wb') as handle:
#     pickle.dump(test_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [116]:
for doc in train_corpus.documents:
    print(doc)
    break

{'id': 0, 'full_text': "winds and ocean currents are carrying a massive oil slick away \nfrom ecuador's galapico's islands. scientists said wednesday the damage \non the islands to rare plant and animal life could have been worse \nif the slick had continued to wash ashore.", 'lang': 'eng', 'date': '2001-01-24 00:00:00', 'cluster': '40001', 'docno': 'VOA20010124.2000.2135', 'features': {'Lemmas_all': {'l_galapico': 0.32831229132315914, 'l_ecuador': 0.2389661221779755, 'l_slick': 0.480664633953581, 'l_ashore': 0.2700018295029832, 'l_ocean': 0.19811616836560575, 'l_animal': 0.17633028567714515, 'l_island': 0.31235515801921876, 'l_plant': 0.15568347849508368, 'l_wind': 0.15701698112610898, 'l_wash': 0.19395199209469, 'l_rare': 0.16608814573040104, 'l_bad': 0.11518475369160017, 'l_damage': 0.14656328713783834, 'l_oil': 0.15027884429765662, 'l_current': 0.13428093787418266, 'l_life': 0.10650595074542996, 'l_wednesday': 0.10527260435746821, 'l_continue': 0.100181632707379, 'l_away': 0.110716

### Lemma all

In [188]:
# check preprocessing_tdt4.py

### Entities_all 

In [125]:
df_texts = pd.read_csv("df_texts_large_entities.csv")

In [138]:
df_texts_filtered = df_texts[~df_texts.entity.isna()]

In [139]:
vectorizer = TfidfVectorizer(stop_words={"English"})
tfidf_matrix = vectorizer.fit_transform(df_texts_filtered.entity)
feature_names = vectorizer.get_feature_names()

/home/hjian42/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['english'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [140]:
print(len(feature_names))

54788


In [143]:
doc2tfidf = {} # docno, tfidf_dict
docno_set = set(df_en_news.docno)

for doc in range(len(df_texts_filtered)):
#     print(doc)
    docno = df_texts_filtered.docno.values[doc]
    if docno in docno_set:
        feature_index = tfidf_matrix[doc,:].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
        tf_idf_dict = {"e_"+ feature_names[i]: s for (i, s) in tfidf_scores}
        doc2tfidf[docno] = tf_idf_dict

In [145]:
for doc in train_corpus.documents:
    doc['features']["Entities_all"] = doc2tfidf[doc['docno']] if doc['docno'] in doc2tfidf else {}
for doc in test_corpus.documents:
    doc['features']["Entities_all"] = doc2tfidf[doc['docno']] if doc['docno'] in doc2tfidf else {}

In [146]:
for doc in train_corpus.documents:
    print(doc)
    break

{'id': 0, 'full_text': "winds and ocean currents are carrying a massive oil slick away \nfrom ecuador's galapico's islands. scientists said wednesday the damage \non the islands to rare plant and animal life could have been worse \nif the slick had continued to wash ashore.", 'lang': 'eng', 'date': '2001-01-24 00:00:00', 'cluster': '40001', 'docno': 'VOA20010124.2000.2135', 'features': {'Lemmas_all': {'l_galapico': 0.32831229132315914, 'l_ecuador': 0.2389661221779755, 'l_slick': 0.480664633953581, 'l_ashore': 0.2700018295029832, 'l_ocean': 0.19811616836560575, 'l_animal': 0.17633028567714515, 'l_island': 0.31235515801921876, 'l_plant': 0.15568347849508368, 'l_wind': 0.15701698112610898, 'l_wash': 0.19395199209469, 'l_rare': 0.16608814573040104, 'l_bad': 0.11518475369160017, 'l_damage': 0.14656328713783834, 'l_oil': 0.15027884429765662, 'l_current': 0.13428093787418266, 'l_life': 0.10650595074542996, 'l_wednesday': 0.10527260435746821, 'l_continue': 0.100181632707379, 'l_away': 0.110716

In [147]:
with open('./train_dev_final.pickle', 'wb') as handle:
    pickle.dump(train_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./test_final.pickle', 'wb') as handle:
    pickle.dump(test_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [149]:
from datetime import datetime

def str2datetime(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    
train_dates = [str2datetime(d['date']) for d in train_corpus.documents]
test_dates = [str2datetime(d['date']) for d in test_corpus.documents]

In [153]:
sorted(test_dates)[-1]

datetime.datetime(2001, 1, 31, 0, 0)

In [154]:
sorted(train_dates)[-1]

datetime.datetime(2001, 1, 31, 0, 0)